In [1]:
import os
import requests
import pandas as pd

#Import london_bike_stations.csv.
df = pd.read_csv("london_bike_stations.csv")

bike_ids = [] #Stored and used as a variable to join DataFrames.

#Loop to pass Bike coordinates to Foursquare URL.
for index, row in df.iterrows():
    lat = row['latitude']
    long = row['longitude']
    bikeid = row['id'] 
    bike_ids.append(bikeid)

df_bike_id = pd.DataFrame({'bike_id': bike_ids})

#Foursquare API key in an environment variable.
fsq_api_key = os.getenv("FOURSQUARE_API_KEY")
#Function to pass information to Foursquare API:
def get_venues_fsq(latitude, longitude, radius, api_key):
    url = f"https://api.foursquare.com/v3/places/search?ll={latitude},{longitude}&radius={radius}"
    headers = {"Accept": "application/json", "Authorization": fsq_api_key}
    response = requests.get(url, headers=headers)
    return response
#Get results for locations in London UK.
london_location = get_venues_fsq(
    latitude = lat,
    longitude = long,
    radius = 1000,
    api_key = fsq_api_key
)

data = london_location.json()

In [2]:
london_uk_fsq = pd.json_normalize(data['results'])

london_uk_fsq_col = london_uk_fsq[['fsq_id', 'distance', 'name', 'location.formatted_address', 'geocodes.main.latitude', 
                                   'geocodes.main.longitude', 'location.locality','location.region','location.postcode']]

london_uk_fsq_rename_col = london_uk_fsq_col.rename(columns={
    'name': 'name',
    'distance': 'four_distance',
    'location.formatted_address': 'four_address',
    'location.locality': 'four_city',
    'location.region': 'four_region',
    'geocodes.main.latitude': 'latitude',
    'geocodes.main.longitude': 'longitude',
    'location.postcode': 'four_postalcode',
     }
)

london_uk_fsq_rename_col['bike_id'] = df_bike_id['bike_id']


In [3]:
london_uk_fsq_rename_col_dropna = london_uk_fsq_rename_col.dropna()

london_uk_fsq_rename_col_dropna.head() 



,fsq_id,four_distance,name,four_address,latitude,longitude,four_city,four_region,four_postalcode,bike_id
0,4bf831094a67c9285df824cf,63,Quartier Dining Room - Eight Moorgate,"1 Dysart St, London, Greater London, EC2A 2BX",51.521503,-0.083882,London,Greater London,EC2A 2BX,7f3020118e56165ed8b2f61899edb971
2,5840778e18384c5356681695,88,Aviary,"22-25 Finsbury Sq (The Montcalm Royal), London...",51.521751,-0.085563,London,Greater London,EC2A 1DX,26184215d38089fcad213ef222e69780
3,5627d3b9498efef2b189e2a0,173,Flight Club,"2a Worship St, London, Greater London, EC2A 2AH",51.522028,-0.086584,London,Greater London,EC2A 2AH,1eabd7ac8e781befd03f52ef56a18aa7
4,5698f64c498e6a3be418ab85,193,The Sichuan,"14 City Rd, London, Greater London, EC1Y 1AU",51.522161,-0.087236,London,Greater London,EC1Y 1AU,23efb32f80a9dcd0e4f61fb44b353ce1
5,5dd505701d1198000826112d,273,Alchemy Coffee,"32a Paul St (Zetland House), London, Greater L...",51.523711,-0.084095,London,Greater London,EC2A 4LF,4c4028fef9930764b34cc63c411cd29f


In [4]:
#Save As .csv file - london_uk_fsq_rename_col_dropna.to_csv("london_uk_fsq_locations.csv")

In [5]:
#Yelp

In [6]:
#Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice.

In [7]:
import requests
import os
import pandas as pd

#Import Bike stations.csv.
df = pd.read_csv("london_bike_stations.csv")

#use variables to join DataFrames.
bike_ids = []
latitudes = []
longitudes = []

#Loop to pass Bike coordinates to Yelp URL.
for index, row in df.iterrows():
    lat = row['latitude']
    latitudes.append(lat)
    long = row['longitude']
    longitudes.append(lat)
    bikeid = row['id']
    bike_ids.append(bikeid)

df_citibike_data = pd.DataFrame({'latitude': latitudes, 'longitude': longitudes, 'bike_id': bike_ids})

#Yelp API key in an environment variable.
yelp_api_key = os.getenv("API_KEY")
#Function to pass information to Foursquare API:
def get_venues_yelp(latitude, longitude, radius, api_key):
    url = f"https://api.yelp.com/v3/businesses/search?latitude={latitude}&longitude={longitude}&radius={radius}"
    headers = {"Accept": "application/json", "Authorization": f"Bearer {api_key}"}
    response = requests.get(url, headers=headers)
    return response

london_location = get_venues_yelp(
    latitude = lat, 
    longitude = long, 
    radius = 1000,
    api_key = ('API_KEY')
)

data = london_location.json()

In [8]:
#Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [9]:
data = london_location.json()


london_uk_yelp = pd.json_normalize(data['businesses'])

london_uk_yelp_col = london_uk_yelp[['id', 'name', 'review_count', 'rating', 'location.display_address']]

london_uk_yelp_rename_col = london_uk_yelp_col.rename(columns={
    'id': 'yelp_id',
    'name': 'yelp_name',
    'review_count': 'yelp_review_count',
    'rating': 'yelp_rating',
    'location.display_address': 'yelp_address'
    }
)

london_uk_yelp_rename_col_add = pd.concat([london_uk_yelp_rename_col, df_citibike_data], axis=1)

In [10]:
#Put your parsed results into a DataFrame

In [11]:
london_uk_yelp_rename_col_add.head()

,yelp_id,yelp_name,yelp_review_count,yelp_rating,yelp_address,latitude,longitude,bike_id
0,P33NBF3mTAgFYLmtMljuzg,Flight Club,34.0,4.3,"[Alphabeta Building, 2A Worship Street, London...",51.529163,51.529163,7f3020118e56165ed8b2f61899edb971
1,hOLFwKg4-0FsCleJ4yoCXw,Duck & Waffle,889.0,4.0,"[110 Bishopsgate, 40th Floor, London EC2N 4AY,...",51.499607,51.499607,67e6c16bce05410ba4b1f0f5000726ea
2,vlq4D26p_143Tj21OF-iMA,Dishoom,666.0,4.5,"[7 Boundary Street, London E2 7JE, United King...",51.505974,51.505974,26184215d38089fcad213ef222e69780
3,vvE5E6WcPkK6ZB3Ivkqiqg,Flat Iron,56.0,4.6,"[77 Curtain Road, Shoreditch EC2A 3BS, United ...",51.523951,51.523951,1eabd7ac8e781befd03f52ef56a18aa7
4,2G0a1v9ubpNO_wADUVgs3A,Yauatcha City,40.0,3.9,"[Broadgate Circle, London EC2M 2QS, United Kin...",51.521681,51.521681,23efb32f80a9dcd0e4f61fb44b353ce1


In [12]:
#Comparing Results

In [13]:
# Which API provided you with more complete data? Provide an explanation.

#The Yelp API yielded  more data has it produced more records than foursquare

#Alongside location details, it also furnished additional information such as ratings and review counts,

#which may serve as variables for subsequent statistical modeling in the project.

#Get the top 10 restaurants according to their rating

In [14]:
top_10_yelp = london_uk_yelp_rename_col_add.sort_values(by='yelp_rating', ascending=False) #Rank Yelp ratings by descending order.

top_10_yelp[['yelp_name', 'yelp_rating']].head(10)

,yelp_name,yelp_rating
6,Blacklock,4.9
3,Flat Iron,4.6
2,Dishoom,4.5
7,Hawksmoor Spitalfields,4.4
8,The Clove Club,4.4
17,Gunpowder,4.4
11,Ozone Coffee,4.4
13,PizzaBuzz,4.4
0,Flight Club,4.3
5,Ottolenghi,4.3


In [15]:
# Create DataFrame from the parsed results
df2 = pd.DataFrame(london_uk_yelp)

df2.to_csv('london_uk_yelp.csv')



In [16]:
london_uk_yelp.head(5)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address,attributes.business_temp_closed,attributes.menu_url,attributes.open24_hours,attributes.waitlist_reservation
0,P33NBF3mTAgFYLmtMljuzg,flight-club-london,Flight Club,https://s3-media4.fl.yelpcdn.com/bphoto/EtViGM...,False,https://www.yelp.com/biz/flight-club-london?ad...,34,"[{'alias': 'pubs', 'title': 'Pubs'}]",4.3,[],...,,London,EC2A 2AH,GB,XGL,"[Alphabeta Building, 2A Worship Street, London...",None,None,None,None
1,hOLFwKg4-0FsCleJ4yoCXw,duck-and-waffle-london,Duck & Waffle,https://s3-media2.fl.yelpcdn.com/bphoto/C8vf46...,False,https://www.yelp.com/biz/duck-and-waffle-londo...,889,"[{'alias': 'modern_european', 'title': 'Modern...",4.0,[],...,,London,EC2N 4AY,GB,XGL,"[110 Bishopsgate, 40th Floor, London EC2N 4AY,...",None,http://duckandwaffle.com/menus,None,None
2,vlq4D26p_143Tj21OF-iMA,dishoom-london-2,Dishoom,https://s3-media2.fl.yelpcdn.com/bphoto/fqlumn...,False,https://www.yelp.com/biz/dishoom-london-2?adju...,666,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...",4.5,[],...,,London,E2 7JE,GB,XGL,"[7 Boundary Street, London E2 7JE, United King...",None,http://www.dishoom.com/wp-content/uploads/2014...,None,None
3,vvE5E6WcPkK6ZB3Ivkqiqg,flat-iron-shoreditch,Flat Iron,https://s3-media3.fl.yelpcdn.com/bphoto/Y126dV...,False,https://www.yelp.com/biz/flat-iron-shoreditch?...,56,"[{'alias': 'steak', 'title': 'Steakhouses'}]",4.6,[],...,None,Shoreditch,EC2A 3BS,GB,XGL,"[77 Curtain Road, Shoreditch EC2A 3BS, United ...",None,None,None,None
4,2G0a1v9ubpNO_wADUVgs3A,yauatcha-city-london-2,Yauatcha City,https://s3-media1.fl.yelpcdn.com/bphoto/6lZcEv...,False,https://www.yelp.com/biz/yauatcha-city-london-...,40,"[{'alias': 'dimsum', 'title': 'Dim Sum'}]",3.9,[],...,,London,EC2M 2QS,GB,XGL,"[Broadgate Circle, London EC2M 2QS, United Kin...",None,https://www.yauatcha.com/city/all-menus/,None,None


In [17]:
# Calculate the number of records obtained from each API
num_records_yelp = len(london_uk_yelp_rename_col)
num_records_foursquare = len(london_uk_fsq_rename_col)

print("Number of Records from Yelp:", num_records_yelp)
print("Number of Records from Foursquare:", num_records_foursquare)

# Assess the completeness of information provided by each API
attributes_yelp = london_uk_yelp_rename_col.columns
attributes_foursquare = london_uk_fsq_rename_col.columns

print("Yelp Attributes:", attributes_yelp)
print("Foursquare Attributes:", attributes_foursquare)



Number of Records from Yelp: 20
Number of Records from Foursquare: 10
Yelp Attributes: Index(['yelp_id', 'yelp_name', 'yelp_review_count', 'yelp_rating',
       'yelp_address'],
      dtype='object')
Foursquare Attributes: Index(['fsq_id', 'four_distance', 'name', 'four_address', 'latitude',
       'longitude', 'four_city', 'four_region', 'four_postalcode', 'bike_id'],
      dtype='object')
